<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/graphSAGE_NN_learn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 47.2 MB/s eta 0:00:00


## 1. A GraphSAGE graph neural network example for homogenious Graph's node classification

In [3]:
import dgl

dataset = dgl.data.CiteseerGraphDataset() ## get the Citerseer dataset
graph = dataset[0]
print(graph)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(3703,), dtype=torch.float32)}
      edata_schemes={})


In [4]:
print(type(graph))

<class 'dgl.heterograph.DGLGraph'>


In [5]:
# 构建一个2层的GNN模型
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        # 实例化SAGEConve，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # 输入是节点的特征
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [6]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [7]:
def evaluate(model, graph, features, labels, mask):
    model.eval() # Sets the model to evaluation mode. In PyTorch, this is important because it disables operations like dropout,
    # which are usually active during training but should be turned off during evaluation.
    # This context manager ensures that during the following block of code, PyTorch won't track operations for gradient computation.
    # This is done for efficiency during evaluation since gradients are not needed.
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [8]:
import torch
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20): ## For showing example, we set 20; user can set 300 for better performance
  # Puts the model in training mode. This is necessary because some layers,
  # like dropout or batch normalization, behave differently during training compared to evaluation.
    model.train() # set model to training mode: compute the grad autoly for parameters
    # 使用所有节点(全图)进行前向传播计算
    logits = model(graph, node_features)
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # 计算验证集的准确度
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # 进行反向传播计算
    # Clears the gradients of all optimized tensors. This is necessary before computing the gradients for the next minibatch.
    opt.zero_grad()
    # Backward pass to compute the gradients of the loss with respect to the model parameters.
    loss.backward()
    # Updates the model parameters based on the computed gradients using the optimization algorithm (assumed to be stored in the opt variable).
    opt.step()
    # Prints the value of the training loss for the current epoch.
    if(epoch % 50 ==0):
      print(loss.item())

    # 如果需要的话，保存训练好的模型。本例中省略。


1.7959935665130615


In [9]:
print(loss) #
print(loss.item())
print(evaluate(model, graph, node_features, node_labels, valid_mask))


tensor(1.4707, grad_fn=<NllLossBackward0>)
1.4707082509994507
0.574


In [10]:
## Test
evaluate(model, graph, node_features, node_labels, test_mask)

0.549

Note: some problems:
(1) how to increase the stacked layers with multiple SAGEConv layers
(2) how to obtain the node embedding after training?
(3) Add the output information for the loss on testing data


## 1.1 Increase stacked layers by designing new

Note: some problems:
(1) how to increase the stacked layers with multiple SAGEConv layers
(2) how to obtain the node embedding after training?
(3) Add the output information for the loss on testing data


In [21]:
class gnn_me(nn.Module):
  def __init__(self, in_feats, hid_feats, out_feats, n_layers):
    super().__init__()
    self.n_layers = n_layers # The layers of NN
    self.layers = nn.ModuleList()
    self.layers.append(SAGE(in_feats=in_feats, out_feats=hid_feats, hid_feats=hid_feats))
    ## Append layers
    for _ in range(n_layers - 1): # Set the last n-1 layers, hide layers use the same number of nerons: n_hidden
            self.layers.append(SAGE(in_feats=hid_feats, out_feats=hid_feats, hid_feats=hid_feats))
    self.linear = nn.Linear(hid_feats, out_feats)
  def forward(self, graph, inputs):
        # 输入是节点的特征
        h = inputs
        for i, layer in enumerate(self.layers):
          h = self.layers[i].forward(graph, h)
        h = self.linear(h)
        return h
  def get_node_embed(self, graph, inputs):
        # 输入是节点的特征
        h = inputs
        for i, layer in enumerate(self.layers):
          h = self.layers[i].forward(graph, h)
        return h

In [22]:
import torch
model = gnn_me(in_feats=n_features, hid_feats=100, out_feats=n_labels, n_layers=2)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20): ## For showing example, we set 20; user can set 300 for better performance
  # Puts the model in training mode. This is necessary because some layers,
  # like dropout or batch normalization, behave differently during training compared to evaluation.
    model.train() # set model to training mode: compute the grad autoly for parameters
    # 使用所有节点(全图)进行前向传播计算
    logits = model(graph, node_features)
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # 计算验证集的准确度
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # 进行反向传播计算
    # Clears the gradients of all optimized tensors. This is necessary before computing the gradients for the next minibatch.
    opt.zero_grad()
    # Backward pass to compute the gradients of the loss with respect to the model parameters.
    loss.backward()
    # Updates the model parameters based on the computed gradients using the optimization algorithm (assumed to be stored in the opt variable).
    opt.step()
    # Prints the value of the training loss for the current epoch.
    if(epoch % 10 ==0):
      print(loss.item())

    # 如果需要的话，保存训练好的模型。本例中省略。


1.803070068359375
1.4888160228729248


In [13]:
print(loss) #
print(loss.item())
print(evaluate(model, graph, node_features, node_labels, valid_mask))

tensor(0.0001, grad_fn=<NllLossBackward0>)
0.00010966437548631802
0.646


In [15]:
##
evaluate(model, graph, node_features, node_labels, test_mask)

0.624

In [30]:
print(graph.ntypes)

['_N']


## 1.2 Get the node embedding after training


In [17]:
## We only add an attribute function in gnn_me class: get_node_embed()
model(graph, node_features).shape

torch.Size([3327, 6])

In [24]:
model.get_node_embed(graph, node_features).shape

torch.Size([3327, 100])

## 1.3 Add the output information for the loss on testing data

In [31]:
import torch
seed = 2024 # set random seed to ensure reproducible
torch.manual_seed(seed)
# If using GPU, set random seed for GPU as well
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
model = gnn_me(in_feats=n_features, hid_feats=100, out_feats=n_labels, n_layers=2)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20): ## For showing example, we set 20; user can set 300 for better performance
  # Puts the model in training mode. This is necessary because some layers,
  # like dropout or batch normalization, behave differently during training compared to evaluation.
    model.train() # set model to training mode: compute the grad autoly for parameters
    # 使用所有节点(全图)进行前向传播计算
    logits = model(graph, node_features)
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # 计算验证集的准确度
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # 计算测试集的准确度
    acc_test = evaluate(model, graph, node_features, node_labels, test_mask)
    # 进行反向传播计算
    # Clears the gradients of all optimized tensors. This is necessary before computing the gradients for the next minibatch.
    opt.zero_grad()
    # Backward pass to compute the gradients of the loss with respect to the model parameters.
    loss.backward()
    # Updates the model parameters based on the computed gradients using the optimization algorithm (assumed to be stored in the opt variable).
    opt.step()
    # Prints the value of the training loss for the current epoch.
    if(epoch % 10 ==0):
      print(f'training loss is: {loss.item(): .3f}, ', f'validation accuracy is: {acc: .2f}, ', f'testing accuracy is: {acc_test: .2f}')

    # 如果需要的话，保存训练好的模型。本例中省略。


training loss is:  1.800,  validation accuracy is:  0.13,  testing accuracy is:  0.16
training loss is:  1.518,  validation accuracy is:  0.55,  testing accuracy is:  0.54


## 1.1.1 Create graph from external sources


In [29]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
g2 = dgl.from_scipy(spmat)
print(g2)
print(type(g2))
print(g2.ntypes) # output is ['_N']：it is a homo graph

Graph(num_nodes=100, num_edges=500,
      ndata_schemes={}
      edata_schemes={})
<class 'dgl.heterograph.DGLGraph'>
['_N']


In [32]:
print(sp.rand(10, 10, density=0.05))
g2.ndata

  (9, 7)	0.11506039163704085
  (3, 1)	0.2061782611085241
  (8, 2)	0.009909055485174179
  (2, 3)	0.010557253855524196
  (0, 9)	0.6131552035770037


{}

### Problem: how to create the heter graph from sparse matrix and add edge features

In [28]:
import dgl
import torch as th

# 创建一个具有3种节点类型和3种边类型的异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
print(g.ntypes) # it is heter. graph
print(type(g))

['disease', 'drug', 'gene']
<class 'dgl.heterograph.DGLGraph'>


## 1.4 How to achieve cell annotation using scRNA-seq data by following scDeepSort

In [32]:
## Load data
import requests
import gzip
from io import BytesIO

# GitHub raw file URL
github_raw_url = 'https://github.com/ZJUFanLab/scDeepSort/raw/master/test/mouse/mouse_Testis199_data.gz'

# Download the compressed file
response = requests.get(github_raw_url)

# Check if the request was successful
if response.status_code == 200:
    # Decompress the content
    compressed_data = BytesIO(response.content)
    with gzip.GzipFile(fileobj=compressed_data, mode='rb') as f:
        decompressed_content = f.read()

    # Now you can work with the decompressed content
    print(decompressed_content.decode('utf-8'))
else:
    print(f"Failed to download file. Status code: {response.status_code}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [52]:
type(decompressed_content)

bytes

In [2]:
## Load data
import requests
import gzip
from io import BytesIO
import numpy as np
from scipy.sparse import csr_matrix

# GitHub raw file URL
github_raw_url = 'https://github.com/ZJUFanLab/scDeepSort/raw/master/test/mouse/mouse_Testis199_data.gz'

# Download the compressed file
response = requests.get(github_raw_url)
# Check if the request was successful
if response.status_code == 200:
    # Decompress the content
    compressed_data = BytesIO(response.content)
    with gzip.GzipFile(fileobj=compressed_data, mode='rb') as f:
        # Read the decompressed content as a numerical matrix with row and column names
        data = np.genfromtxt(f, delimiter='\t', dtype=None, names=None, encoding=None)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

In [85]:
print(type(data))
data.shape
print(data)


<class 'numpy.ndarray'>
['"","C_1","C_2","C_3","C_4","C_5","C_6","C_7","C_8","C_9","C_10","C_11","C_12","C_13","C_14","C_15","C_16","C_17","C_18","C_19","C_20","C_21","C_22","C_23","C_24","C_25","C_26","C_27","C_28","C_29","C_30","C_31","C_32","C_33","C_34","C_35","C_36","C_37","C_38","C_39","C_40","C_41","C_42","C_43","C_44","C_45","C_46","C_47","C_48","C_49","C_50","C_51","C_52","C_53","C_54","C_55","C_56","C_57","C_58","C_59","C_60","C_61","C_62","C_63","C_64","C_65","C_66","C_67","C_68","C_69","C_70","C_71","C_72","C_73","C_74","C_75","C_76","C_77","C_78","C_79","C_80","C_81","C_82","C_83","C_84","C_85","C_86","C_87","C_88","C_89","C_90","C_91","C_92","C_93","C_94","C_95","C_96","C_97","C_98","C_99","C_100","C_101","C_102","C_103","C_104","C_105","C_106","C_107","C_108","C_109","C_110","C_111","C_112","C_113","C_114","C_115","C_116","C_117","C_118","C_119","C_120","C_121","C_122","C_123","C_124","C_125","C_126","C_127","C_128","C_129","C_130","C_131","C_132","C_133","C_134","C_135"

In [83]:
## Prepare data for defining function
data_tmp = data[1]
tmp1 = data[1].split(',')
gene_tmp = tmp1[0]
gexp_tmp = tmp1[1:]
print(gene_tmp)
print(gexp_tmp)
print(len(gexp_tmp))
numerical_values = [float(value) for value in gexp_tmp]
print(numerical_values)
type(numerical_values)

"0610009B22Rik"
['0.593387713582016', '1.06162810438859', '0', '2.62437183148919', '0', '2.01347435610029', '0', '1.95038058291025', '0', '2.10090958871987', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2.71366819340643', '0', '0', '3.58765993285183', '3.35692607791887', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '3.37310698435427', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '3.5657017386646', '0', '4.21788332934208', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '4.32581158608494', '0', '0', '0', '0', '0', '0'

list

In [3]:
## Define a function to tranfer all string to numerical values
def myfunc(str_value):
  tmp1 = str_value.split(',')
  gene_tmp = tmp1[0].replace('"', '')
  gexp_tmp = tmp1[1:]
  numerical_values = [float(value) for value in gexp_tmp]
  return [gene_tmp, numerical_values]
myfunc(data[1])[0]

'0610009B22Rik'

'0610009B22Rik'

In [4]:
matrix = []
gene_names = []
# Iterate over the lists and append each list to the matrix
for i in range(len(data)-1):
    list_tmp = myfunc(data[i+1])
    gene_names.append(list_tmp[0])
    matrix.append(list_tmp[1])

# Convert the list of lists to a NumPy array
matrix = np.array(matrix)

In [5]:
#print(matrix)
matrix.shape
gene_names[0:2]
cell_names = data[0].split(',')[1:]
print(cell_names[:3])
print(len(cell_names))

['"C_1"', '"C_2"', '"C_3"']
199


In [6]:
import numpy as np
from scipy.sparse import csr_matrix
exp_mat = csr_matrix(matrix)


In [7]:
# Read cell type from github

import pandas as pd

# GitHub raw CSV file URL
github_raw_url = 'https://github.com/ZJUFanLab/scDeepSort/raw/master/test/mouse/mouse_Testis199_celltype.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(github_raw_url)

# Display the DataFrame
print(df)


     Unnamed: 0   Cell     Cell_type
0             1    C_1  Spermatocyte
1             2    C_2  Spermatocyte
2             3    C_3  Spermatocyte
3             4    C_4  Spermatocyte
4             5    C_5  Spermatocyte
..          ...    ...           ...
194         195  C_195  Spermatocyte
195         196  C_196  Spermatocyte
196         197  C_197  Spermatocyte
197         198  C_198  Spermatocyte
198         199  C_199  Spermatocyte

[199 rows x 3 columns]


In [8]:
df['Cell_type'].map(str.strip)
set(df.values[:, 2])

{'Spermatocyte'}

In [10]:
num_genes = len(gene_names)
print(num_genes)

9339


In [81]:
import dgl
import torch
device = 'cpu'
graph = dgl.DGLGraph() # start to tranfer data to a graph
    # graph = dgl.DGLGraph(): This creates an empty directed graph using DGL. DGLGraph is a class in DGL that represents a graph.
    # At this point, the graph has no nodes or edges.
gene_ids = torch.arange(num_genes, dtype=torch.int32, device=device).unsqueeze(-1)
print(gene_ids)

tensor([[   0],
        [   1],
        [   2],
        ...,
        [9336],
        [9337],
        [9338]], dtype=torch.int32)


/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [82]:
graph.add_nodes(num_genes, {'id': gene_ids}) ## add nodes and node feature with 1-dim
print(graph)
graph.ndata['id']

Graph(num_nodes=9339, num_edges=0,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={})


tensor([[   0],
        [   1],
        [   2],
        ...,
        [9336],
        [9337],
        [9338]], dtype=torch.int32)

In [71]:
9339*2
g2 = dgl.DGLGraph()
print(g2)
g2_ids = torch.arange(num_genes, dtype=torch.int32, device=device).unsqueeze(-1)
g2.add_nodes(num_genes, {'id': g2_ids})
print(g2)

Graph(num_nodes=0, num_edges=0,
      ndata_schemes={}
      edata_schemes={})
Graph(num_nodes=9339, num_edges=0,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={})


/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [23]:
github_raw_url = 'https://github.com/ZJUFanLab/scDeepSort/raw/master/test/mouse/mouse_Testis199_data.gz'
df_exp = pd.read_csv(github_raw_url, compression='gzip', index_col=0)

In [31]:
#df_exp
df_exp = df_exp.T # (cell, gene)
df_exp

,0610009B22Rik,0610009L18Rik,0610010F05Rik,0610012G03Rik,0610030E20Rik,0610040B10Rik,1010001B22Rik,1110002J07Rik,1110002L01Rik,1110004F10Rik,...,Nsd2,Nsd3,Map3k20,Kbtbd6,Tut4,Tut7,Zfp638,Zfp607a,Zfp512b,Zup1
C_1,0.593388,1.444667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.593388,...,0.000000,0.0,0.0,0.000000,0.593388,0.593388,0.000000,0.593388,0.000000,0.000000
C_2,1.061628,2.147602,1.564890,1.061628,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.061628,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.564890,2.147602
C_3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.054176,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.481651,0.000000
C_4,2.624372,2.001203,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,2.001203,0.000000,0.000000,0.000000,0.000000,2.001203,0.000000
C_5,0.000000,0.000000,1.774064,0.000000,0.0,0.0,0.0,0.0,0.0,1.774064,...,1.774064,0.0,0.0,0.000000,0.000000,0.000000,2.378576,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_195,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C_196,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C_197,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C_198,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
gene2id = {gene: idx for idx, gene in enumerate(gene_names)}
# filter out cells not in label-text
#df_exp = df_exp.iloc[filter_cell]
df_exp = df_exp.rename(columns=gene2id)

In [36]:
# filter out useless columns if exists (when using gene intersection)
col = [c for c in df_exp.columns if c in gene2id.values()]
df_exp = df_exp[col]
print(col)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [83]:
graph.number_of_nodes()

9339

In [86]:
arr = df_exp.to_numpy() # transfer dataframe to numpy matrix
arr.shape
threshold = 0.01
row_idx, col_idx = np.nonzero(arr > threshold)  # intra-dataset index
non_zeros = arr[(row_idx, col_idx)]  # non-zero values
print(non_zeros)
print(row_idx)
cell_idx = row_idx + graph.number_of_nodes()  # cell_index
print(len(cell_idx))
print(max(cell_idx))
gene_idx = df_exp.columns[col_idx].astype(int).tolist()  # gene_index
print(f'length: {len(gene_idx)}, ', f'gene index: {gene_idx}')
info_shape = (len(df_exp), num_genes)
print(info_shape)

[0.59338771 1.4446672  0.59338771 ... 3.7907599  3.7907599  3.7907599 ]
[  0   0   0 ... 198 198 198]
75168
9537
length: 75168,  gene index: [0, 1, 9, 11, 15, 16, 18, 19, 21, 23, 30, 34, 35, 37, 38, 40, 41, 42, 44, 45, 46, 47, 49, 51, 52, 53, 54, 55, 56, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 107, 108, 117, 119, 121, 122, 123, 124, 125, 126, 129, 130, 131, 136, 137, 140, 141, 144, 146, 147, 149, 150, 151, 152, 153, 154, 155, 158, 159, 161, 164, 165, 167, 169, 171, 172, 173, 175, 177, 178, 179, 181, 182, 183, 184, 185, 187, 189, 190, 192, 193, 195, 196, 199, 201, 202, 204, 208, 209, 210, 211, 213, 214, 215, 218, 219, 222, 226, 228, 239, 244, 245, 247, 250, 257, 262, 264, 265, 269, 270, 271, 272, 274, 277, 282, 283, 284, 285, 286, 290, 292, 293, 296, 297, 299, 300, 301, 303, 304, 305, 306, 307, 309, 310, 311, 313, 315, 316, 317, 318, 319, 321, 322, 323, 328, 329, 331, 333, 334, 336, 337,

In [58]:
info = csr_matrix((non_zeros, (row_idx, gene_idx)), shape=info_shape)
print(info.shape)
# use -1 to denote cell
ids = torch.tensor([-1] * len(df), dtype=torch.int32, device=device).unsqueeze(-1)

(199, 9339)


In [88]:
graph.add_nodes(len(df_exp), {'id': ids})
print(graph)
graph.ndata['id'].shape ## add a node feature named id
graph.ndata['id'][-1]

Graph(num_nodes=9538, num_edges=0,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={})


tensor([-1], dtype=torch.int32)

In [87]:
print(len(cell_idx))
print(len(gene_idx))
print(max(gene_idx))
print(max(cell_idx))

75168
75168
9338
9537


In [89]:
graph.add_edges(cell_idx, gene_idx, ## Start adding weights of the expression values
        {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})
graph.add_edges(gene_idx, cell_idx,
        {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})
print(graph)

Graph(num_nodes=9538, num_edges=150336,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float32)})


In [90]:
199+9339

9538

In [95]:
print(f'Added {len(df_exp)} nodes and {len(cell_idx)} edges.')
print(f'#Nodes in Graph: {graph.number_of_nodes()}, #Edges: {graph.number_of_edges()}.')
graph.in_degrees()

Added 199 nodes and 75168 edges.
#Nodes in Graph: 9538, #Edges: 150336.


tensor([ 15,  48,  14,  ..., 208, 207, 208])

In [97]:
def normalize_weight(graph: dgl.DGLGraph):
    # normalize weight & add self-loop
    in_degrees = graph.in_degrees()
    for i in range(graph.number_of_nodes()):
        src, dst, in_edge_id = graph.in_edges(i, form='all')
        if src.shape[0] == 0:
            continue
        edge_w = graph.edata['weight'][in_edge_id]
        graph.edata['weight'][in_edge_id] = in_degrees[i] * edge_w / torch.sum(edge_w)

In [99]:
# normalize weight
normalize_weight(graph)
# add self-loop
graph.add_edges(graph.nodes(), graph.nodes(),
        {'weight': torch.ones(graph.number_of_nodes(), dtype=torch.float, device=device).unsqueeze(1)})
print(graph)

Graph(num_nodes=9538, num_edges=169412,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float32)})
